In [ ]:
%%script false --no-raise-error

select
  users.id as stash_user_id,
  min(users.created_at) as create_,
  min(subscription_actions.created_at) as tier_,
  min(funding_sources.created_at) as plastic_,
  min(user_esignatures.esigned_at) as advisory_,
  min(users.e_signature_agreed_at) as accounts_,
  min(account_state_transitions.created_at) as complete_,
  min(user_pmt_linked.linked_bank_funding_date) as bank_,
  min(invoices.charge_date) as attempt_,
  min(invoices.collection_completed_at) as collection_,
  case
    when datediff(minute,create_,bank_) <= 15 and
      datediff(minute,create_,complete_) <= 15 then '7_complete_and_bank_link'
    when datediff(minute,create_,complete_) <= 15 then '6_complete'
    when datediff(minute,create_,accounts_) <= 15 then '5_accounts_esig'
    when datediff(minute,create_,advisory_) <= 15 then '4_advisory_esig'
    when datediff(minute,create_,plastic_) <= 15 then '3_plastic_link'
    when datediff(minute,create_,tier_) <= 15 then '2_tier_select'
    else '1_user_create' end as funnel_depth_15_min,
  funnel_depth_15_min in ('1_user_create','2_tier_select','3_plastic_link','4_advisory_esig')
    as rescue_entrant,
  case
    when datediff(minute,create_,bank_) <= 15 + 8*1440 and
      datediff(minute,create_,complete_) <= 15 + 8*1440 then '7_complete_and_bank_link'
    when datediff(minute,create_,complete_) <= 15 + 8*1440 then '6_complete'
    when datediff(minute,create_,accounts_) <= 15 + 8*1440 then '5_accounts_esig'
    when datediff(minute,create_,advisory_) <= 15 + 8*1440 then '4_advisory_esig'
    when datediff(minute,create_,plastic_) <= 15 + 8*1440 then '3_plastic_link'
    when datediff(minute,create_,tier_) <= 15 + 8*1440 then '2_tier_select'
    else '1_user_create' end as funnel_depth_8_days,
  case
    when bank_ is not null and complete_ is not null then '7_complete_and_bank_link'
    when complete_ is not null then '6_complete'
    when accounts_ is not null then '5_accounts_esig'
    when advisory_ is not null then '4_advisory_esig'
    when plastic_ is not null then '3_plastic_link'
    when tier_ is not null then '2_tier_select'
    else '1_user_create' end as funnel_depth,
  case
    when collection_ is not null then 'D_collected'
    when attempt_ is not null then 'C_attempted'
    when advisory_ is not null then 'B_everchargeable'
    else 'A_neverchargeable' end as chargeable_funnel /* OB2 only */
from
  source_pg_main.users left join
  source_subscriptions.users as subscription_users on users.uuid = subscription_users.uuid left join
  source_subscriptions.user_esignatures on            subscription_users.id = user_esignatures.user_id left join
  source_subscriptions.subscriptions on               subscription_users.id = subscriptions.user_id left join
  source_subscriptions.invoices on                    subscriptions.id = invoices.subscription_id left join
  source_subscriptions.subscription_actions on        subscriptions.id = subscription_actions.subscription_id and
                                                      subscription_actions.new_plan_id is not null and
                                                      subscription_actions.action = 'create' left join
  integrated_users.user_pmt_linked on                 users.id = user_pmt_linked.user_id left join
  source_pg_main.funding_sources on                   users.id = funding_sources.user_id and
                                                      funding_sources.provider_type like '%PLASTIC%' left join
  source_pg_main.accounts on                          users.id = accounts.user_id left join
  source_pg_main.account_state_transitions on         accounts.id = account_state_transitions.account_id and
                                                      account_state_transitions.to_aasm_state = 'complete'
where users.created_at > '2021-07-01'
group by 1

In [ ]:
%%script false --no-raise-error

view: first_funnel_depth {
  derived_table: {
    sql:
        select
          users.id as stash_user_id,
          min(users.created_at) as create_,
          min(subscription_actions.created_at) as tier_,
          min(funding_sources.created_at) as plastic_,
          min(user_esignatures.esigned_at) as advisory_,
          min(users.e_signature_agreed_at) as accounts_,
          min(account_state_transitions.created_at) as complete_,
          min(user_pmt_linked.linked_bank_funding_date) as bank_,
          min(invoices.charge_date) as attempt_,
          min(invoices.collection_completed_at) as collection_,
          case
            when datediff(minute,create_,bank_) <= 15 and
              datediff(minute,create_,complete_) <= 15 then '7_complete_and_bank_link'
            when datediff(minute,create_,complete_) <= 15 then '6_complete'
            when datediff(minute,create_,accounts_) <= 15 then '5_accounts_esig'
            when datediff(minute,create_,advisory_) <= 15 then '4_advisory_esig'
            when datediff(minute,create_,plastic_) <= 15 then '3_plastic_link'
            when datediff(minute,create_,tier_) <= 15 then '2_tier_select'
            else '1_user_create' end as funnel_depth_15_min,
          funnel_depth_15_min in ('1_user_create','2_tier_select','3_plastic_link','4_advisory_esig')
            as rescue_entrant,
          case
            when datediff(minute,create_,bank_) <= 15 + 8*1440 and
              datediff(minute,create_,complete_) <= 15 + 8*1440 then '7_complete_and_bank_link'
            when datediff(minute,create_,complete_) <= 15 + 8*1440 then '6_complete'
            when datediff(minute,create_,accounts_) <= 15 + 8*1440 then '5_accounts_esig'
            when datediff(minute,create_,advisory_) <= 15 + 8*1440 then '4_advisory_esig'
            when datediff(minute,create_,plastic_) <= 15 + 8*1440 then '3_plastic_link'
            when datediff(minute,create_,tier_) <= 15 + 8*1440 then '2_tier_select'
            else '1_user_create' end as funnel_depth_8_days,
          case
            when bank_ is not null and complete_ is not null then '7_complete_and_bank_link'
            when complete_ is not null then '6_complete'
            when accounts_ is not null then '5_accounts_esig'
            when advisory_ is not null then '4_advisory_esig'
            when plastic_ is not null then '3_plastic_link'
            when tier_ is not null then '2_tier_select'
            else '1_user_create' end as funnel_depth,
          case
            when collection_ is not null then 'D_collected'
            when attempt_ is not null then 'C_attempted'
            when advisory_ is not null then 'B_ever_chargeable'
            else 'A_never_chargeable' end as collection_funnel /* OB2 only */
        from
          source_pg_main.users left join
          source_subscriptions.users as subscription_users on users.uuid = subscription_users.uuid left join
          source_subscriptions.user_esignatures on            subscription_users.id = user_esignatures.user_id left join
          source_subscriptions.subscriptions on               subscription_users.id = subscriptions.user_id left join
          source_subscriptions.invoices on                    subscriptions.id = invoices.subscription_id left join
          source_subscriptions.subscription_actions on        subscriptions.id = subscription_actions.subscription_id and
                                                              subscription_actions.new_plan_id is not null and
                                                              subscription_actions.action = 'create' left join
          integrated_users.user_pmt_linked on                 users.id = user_pmt_linked.user_id left join
          source_pg_main.funding_sources on                   users.id = funding_sources.user_id and
                                                              funding_sources.provider_type like '%PLASTIC%' left join
          source_pg_main.accounts on                          users.id = accounts.user_id left join
          source_pg_main.account_state_transitions on         accounts.id = account_state_transitions.account_id and
                                                              account_state_transitions.to_aasm_state = 'complete'
        group by 1
       ;;
  }

  measure: count {
    type: count
    # drill_fields: [detail*]
  }

  dimension: stash_user_id {
    type: number
    primary_key: yes
    sql: ${TABLE}.stash_user_id ;;
  }

  dimension_group: user_create {
    alias: [user_create_at]
    type: time
    sql: ${TABLE}.create_ ;;
    timeframes: [time]
    group_label: "Event Times"
  }
  dimension_group: tier_select {
    alias: [tier_select_at]
    type: time
    sql: ${TABLE}.tier_ ;;
    timeframes: [time]
    group_label: "Event Times"
  }
  dimension_group: plastic_link {
    alias: [plastic_link_at]
    type: time
    sql: ${TABLE}.plastic_ ;;
    timeframes: [time]
    group_label: "Event Times"
  }
  dimension_group: advisory_esig {
    alias: [advisory_esig_at]
    type: time
    sql: ${TABLE}.advisory_ ;;
    timeframes: [time]
    group_label: "Event Times"
  }
  dimension_group: accounts_esig {
    alias: [accounts_esig_at]
    type: time
    sql: ${TABLE}.accounts_ ;;
    timeframes: [time]
    group_label: "Event Times"
  }
  dimension_group: complete {
    alias: [complete_at]
    type: time
    sql: ${TABLE}.complete_ ;;
    timeframes: [time]
    group_label: "Event Times"
  }
  dimension_group: bank_link {
    alias: [bank_link_at]
    type: time
    sql: ${TABLE}.bank_ ;;
    timeframes: [time]
    group_label: "Event Times"
  }

  
  dimension: rescue_entrant {
    type: yesno
    sql: ${TABLE}.rescue_entrant ;;
  }
  dimension: invest_funnel_15_min {
    alias: [funnel_depth_15_min]
    type: string
    sql: ${TABLE}.funnel_depth_15_min ;;
    suggestions: [
      "1_user_create",
      "2_tier_select",
      "3_plastic_link",
      "4_advisory_esig",
      "5_accounts_esig",
      "6_complete",
      "7_complete_and_bank_link"
    ]
    group_label: "Invest Funnel"
  }

  dimension: invest_funnel_8_days {
    alias: [funnel_depth_8_days]
    type: string
    sql: ${TABLE}.funnel_depth_8_days ;;
    suggestions: [
      "1_user_create",
      "2_tier_select",
      "3_plastic_link",
      "4_advisory_esig",
      "5_accounts_esig",
      "6_complete",
      "7_complete_and_bank_link"
    ]
    group_label: "Invest Funnel"
  }
  dimension: invest_funnel {
    alias: [funnel_depth]
    type: string
    sql: ${TABLE}.funnel_depth ;;
    suggestions: [
      "1_user_create",
      "2_tier_select",
      "3_plastic_link",
      "4_advisory_esig",
      "5_accounts_esig",
      "6_complete",
      "7_complete_and_bank_link"
    ]
    group_label: "Invest Funnel"
  }
  dimension: collection_funnel {
    type: string
    description: "OB2 only"
    sql: ${TABLE}.collection_funnel ;;
    suggestions: [
      "A_never_chargeable",
      "B_ever_chargeable",
      "C_attempted",
      "D_collected"
    ]
  }

  # set: detail {
  #   fields: [
  #     stash_user_id,
  #     user_create_time,
  #     tier_select_time,
  #     plastic_link_time,
  #     advisory_esig_time,
  #     accounts_esig_time,
  #     complete_time,
  #     bank_link_time,
  #     funnel_depth_15_min
  #   ]
  # }
}
